DQN Baseline

Logan Wong

law3082

In [1]:
# REMINDER: make sure you set
# Runtime
# Change runtime type
# T4 GPU

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari
!ls -la

/content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari
total 27
drwx------ 2 root root 4096 Sep 25 15:17 checkpoints
drwx------ 2 root root 4096 Sep 25 15:17 code
drwx------ 2 root root 4096 Sep 25 15:04 .git
-rw------- 1 root root 5787 Sep 25 16:02 github_terminal.ipynb
-rw------- 1 root root   22 Sep 25 15:13 .gitignore
drwx------ 2 root root 4096 Sep 25 15:17 models
-rw------- 1 root root   26 Sep 25 15:04 README.md
drwx------ 2 root root 4096 Sep 25 15:17 results


In [4]:
!pip install stable-baselines3 gymnasium[atari,accept-rom-license] ale-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 6.9 MB/s eta 0:00:00


In [5]:
import os
import torch
import gymnasium as gym
import stable_baselines3
import ale_py
import numpy as np
from stable_baselines3 import DQN


# Visualization
from PIL import Image
import io
import base64
from IPython.display import display, HTML



# For debugging
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.callbacks import BaseCallback
import time

# Action masking
# from gymnasium import ActionWrapper

# Vector environment
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, VecEnvWrapper

print("All imports working")

All imports working


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [6]:
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


GPU available: True
GPU: Tesla T4


Create Environment and Model

In [7]:
seed = 316
torch.manual_seed(seed)

env = make_atari_env("BowlingNoFrameskip-v4", n_envs=1, seed=seed)

model = DQN(
    "CnnPolicy",
    env,
    learning_rate=0.0001,
    buffer_size=100000,
    batch_size=32,
    gamma=0.99,
    target_update_interval=1000,
    train_freq=4,
    gradient_steps=1,
    exploration_final_eps=0.01,
    exploration_fraction=0.1,
    learning_starts=100000,
    verbose=1,
    device="cuda"
)

print("DQN model created with GPU support!")

Using cuda device
Wrapping the env in a VecTransposeImage.
DQN model created with GPU support!


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# Action space: Discrete(6)
# Number of actions: 6
# Action meanings: ['NOOP', 'FIRE', 'UP', 'DOWN', 'UPFIRE', 'DOWNFIRE']
# Observation shape: (210, 160, 3)
# Observation space: Box(0, 255, (210, 160, 3), uint8)

# ACTIONS:
# 0: NOOP
# 1: FIRE
# 2: UP
# 3: DOWN
# 4: UPFIRE
# 5: DOWNFIRE

In [9]:
def convert(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60

    return "%d:%02d:%02d" % (hour, minutes, seconds)

In [10]:
# Show the first frame as an image
def show_frame(obs):
    # Convert to PIL Image and display
    img = Image.fromarray(obs)
    display(img)

Train the model

In [11]:
# Save every 500K steps so you never lose more than 30 minutes
# checkpoint_callback = CheckpointCallback(
#     save_freq=10000,
#     save_path="/content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari/checkpoints",
#     name_prefix="dqn_baseline_100K"
# )

total_timesteps = 10000000    # 10M
# total_timesteps =  2000000    # 2M
# total_timesteps =  1000000    # 1M
# total_timesteps =   100000    # 100K
# total_timesteps =    10000    # 10K
# total_timesteps =     5000    # 5K

# Time how long it takes
print("Training started")
start_time = time.time()
# model.learn(total_timesteps=total_timesteps, callback=checkpoint_callback)
model.learn(total_timesteps=total_timesteps, log_interval=10)
end_time = time.time()
print("Training done")

env.close()

# Calculate run time
training_duration = end_time - start_time
time_in_minutes_and_seconds = convert(training_duration)
print(f"Time taken: {time_in_minutes_and_seconds}")
# print(f"Speed: {total_timesteps/training_duration:.2f} steps/second")

# Save model to Google Drive
model.save("/content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari/models/dqn_baseline_100K")
print("Model saved to Google Drive")

Training started
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.46e+03 |
|    ep_rew_mean      | 22.8     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 10       |
|    fps              | 806      |
|    time_elapsed     | 26       |
|    total_timesteps  | 21083    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.46e+03 |
|    ep_rew_mean      | 22.6     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 20       |
|    fps              | 806      |
|    time_elapsed     | 52       |
|    total_timesteps  | 42164    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.47e+03 |
|    ep_rew_mean      | 23.4     |
|    exploration_rate | 0.01     |
| time/               |          |
|  